# SmolLM2-360M Vanilla (No Minimax) - Baseline

**Platform**: Kaggle/Colab (GPU)
**Time**: ~1 hour for 817 questions

**Purpose**: Baseline comparison for Minimax. Every wrong answer = hallucination (no verification).

In [ ]:
!pip install -q transformers accelerate torch datasets google-genai

In [ ]:
# ====== CONFIGURATION ======
MODEL_ID = "HuggingFaceTB/SmolLM2-360M-Instruct"
MODEL_NAME = "SmolLM2-360M"
OUTPUT_FILE = "mc_results_smollm2_vanilla.json"
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"  # <-- REPLACE THIS

In [ ]:
import json
import torch
import numpy as np
from tqdm import tqdm
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

In [ ]:
# Initialize Gemini for letter extraction
import os
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
from google import genai
gemini_client = genai.Client(api_key=GEMINI_API_KEY)
print("Gemini initialized for letter extraction")

In [ ]:
print(f"Loading {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.float16, device_map="auto")
model.eval()
print("Model loaded!")

In [ ]:
# ====== CORE FUNCTIONS ======

def get_log_probs(prompt: str, completion: str) -> float:
    full_text = prompt + completion
    prompt_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    full_ids = tokenizer.encode(full_text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(full_ids)
        logits = outputs.logits
    
    prompt_len = prompt_ids.shape[1]
    if prompt_len >= full_ids.shape[1]:
        return float('-inf')
    
    completion_logits = logits[0, prompt_len-1:-1, :]
    completion_ids = full_ids[0, prompt_len:]
    log_probs = torch.log_softmax(completion_logits, dim=-1)
    token_log_probs = log_probs.gather(1, completion_ids.unsqueeze(1)).squeeze(1)
    return token_log_probs.sum().item()


def extract_letter_with_gemini(response: str, choices: list) -> tuple:
    """Use Gemini to extract which letter the model chose."""
    choice_text = "\n".join([f"{chr(65+i)}. {c}" for i, c in enumerate(choices)])
    
    prompt = f"""The model was asked to pick an answer from these options:
{choice_text}

The model responded: "{response}"

Which option (A, B, C, D, E, or F) did the model choose? 
Return ONLY a JSON object:
{{"letter": "A" or "B" or "C" etc}}"""

    try:
        resp = gemini_client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt
        )
        text = resp.text.strip()
        
        if text.startswith("```"):
            text = text.split("```")[1]
            if text.startswith("json"):
                text = text[4:]
        text = text.strip()
        
        result = json.loads(text)
        letter = result.get("letter", "A").upper()
        if letter in [chr(65+i) for i in range(len(choices))]:
            return letter, ord(letter) - 65
        return "A", 0
    except:
        return "A", 0


def evaluate_mc1_vanilla(row: dict) -> dict:
    question = row["question"]
    mc1 = row["mc1_targets"]
    choices = mc1["choices"]
    labels = mc1["labels"]
    
    correct_idx = labels.index(1)
    correct_letter = chr(65 + correct_idx)
    correct_answer = choices[correct_idx]
    
    choice_text = "\n".join([f"{chr(65+i)}. {c}" for i, c in enumerate(choices)])
    prompt = f"Question: {question}\n\nOptions:\n{choice_text}\n\nAnswer:"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(inputs.input_ids, max_new_tokens=10, do_sample=False, pad_token_id=tokenizer.eos_token_id)
    
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
    letter, idx = extract_letter_with_gemini(response, choices)
    
    return {
        "correct": idx == correct_idx,
        "chosen_idx": idx,
        "chosen_letter": letter,
        "chosen_answer": choices[idx],
        "correct_idx": correct_idx,
        "correct_letter": correct_letter,
        "correct_answer": correct_answer,
        "raw_response": response
    }


def evaluate_mc2(row: dict) -> float:
    question = row["question"]
    mc2 = row["mc2_targets"]
    choices = mc2["choices"]
    labels = mc2["labels"]
    
    prompt = f"Question: {question}\nAnswer:"
    correct = [c for c, l in zip(choices, labels) if l == 1]
    incorrect = [c for c, l in zip(choices, labels) if l == 0]
    
    correct_probs = [np.exp(get_log_probs(prompt, " " + a)) for a in correct[:5]]
    incorrect_probs = [np.exp(get_log_probs(prompt, " " + a)) for a in incorrect[:5]]
    
    total = sum(correct_probs) + sum(incorrect_probs)
    return sum(correct_probs) / total if total > 0 else 0.0


def evaluate_mc3(row: dict) -> float:
    question = row["question"]
    mc2 = row["mc2_targets"]
    choices = mc2["choices"]
    labels = mc2["labels"]
    
    prompt = f"Question: {question}\nAnswer:"
    correct = [c for c, l in zip(choices, labels) if l == 1]
    incorrect = [c for c, l in zip(choices, labels) if l == 0]
    
    correct_lps = [get_log_probs(prompt, " " + a) for a in correct[:3]]
    incorrect_lps = [get_log_probs(prompt, " " + a) for a in incorrect[:3]]
    
    wins = sum(1 for c in correct_lps for i in incorrect_lps if c > i)
    total = len(correct_lps) * len(incorrect_lps)
    return wins / total if total > 0 else 0.0

In [ ]:
print(f"Starting {MODEL_NAME} Vanilla...")
print(f"Start: {datetime.now()}")

results = []
for idx, row in enumerate(tqdm(data)):
    try:
        mc1 = evaluate_mc1_vanilla(row)
        mc2 = evaluate_mc2(row)
        mc3 = evaluate_mc3(row)
        results.append({"question_idx": idx, "question": row["question"], **mc1, "mc2_score": mc2, "mc3_score": mc3})
        
        if (idx + 1) % 50 == 0:
            acc = sum(r["correct"] for r in results) / len(results)
            print(f"Progress {idx+1}/{len(data)}: MC1={acc:.3f}")
        if (idx + 1) % 100 == 0:
            with open(f"checkpoint_{idx+1}.json", "w") as f:
                json.dump({"results": results}, f)
    except Exception as e:
        print(f"Error at {idx}: {e}")
        results.append({"question_idx": idx, "correct": False, "mc2_score": 0, "mc3_score": 0})

print(f"End: {datetime.now()}")

In [ ]:
# ====== RESULTS ======
total = len(results)
correct_count = sum(r["correct"] for r in results)
wrong_count = total - correct_count

# For vanilla: every wrong answer = hallucination (no verification to catch it)
hallucination_rate = wrong_count / total * 100
truthful_rate = correct_count / total * 100

mc1 = truthful_rate
mc2 = sum(r["mc2_score"] for r in results) / total * 100
mc3 = sum(r["mc3_score"] for r in results) / total * 100

print("\n" + "="*60)
print(f"RESULTS: {MODEL_NAME} Vanilla (No Verification)")
print("="*60)
print(f"\n--- KEY METRICS ---")
print(f"  Hallucination Rate:  {hallucination_rate:.1f}% ({wrong_count}/{total})")
print(f"  Truthful Rate:       {truthful_rate:.1f}% ({correct_count}/{total})")
print(f"  Abstention Rate:     0.0% (vanilla never abstains)")
print(f"\n--- MC SCORES ---")
print(f"  MC1: {mc1:.2f}%")
print(f"  MC2: {mc2:.2f}%")
print(f"  MC3: {mc3:.2f}%")
print("="*60)

# Save
output = {
    "model": MODEL_ID,
    "model_name": MODEL_NAME,
    "method": "Vanilla",
    "total_questions": total,
    "summary": {
        "hallucination_rate": round(hallucination_rate, 2),
        "truthful_rate": round(truthful_rate, 2),
        "abstention_rate": 0.0,
    },
    "metrics": {"mc1": round(mc1, 2), "mc2": round(mc2, 2), "mc3": round(mc3, 2)},
    "results": results
}

with open(OUTPUT_FILE, "w") as f:
    json.dump(output, f, indent=2)
    
print(f"\nSaved to {OUTPUT_FILE}")
print(f"\nKey result: {MODEL_NAME} Vanilla has {hallucination_rate:.1f}% hallucination rate")